In [7]:
import requests
url = "https://op.mos.ru/EHDWSREST/catalog/export/get?id=331224"
response = requests.get(url)
if response.status_code == 200:
    with open("kovorkingi.zip", 'wb') as f:
        f.write(response.content)

In [8]:
import zipfile
with zipfile.ZipFile("kovorkingi.zip", 'r') as zip_ref:
    zip_ref.extractall(".")

In [9]:
import json

with open("data-4209-2018-09-11.json", 'rb') as fd:
    data = json.loads(fd.read().decode("cp1251"))

In [10]:
import pandas as pd
df = pd.io.json.json_normalize(data)
df.head(2)

,Address,AdmArea,ContactPhone,District,Email,FullName,ID,LocationClarification,ShortDescription,ShortName,Site,Specialization,Tariffs,Year,geoData.coordinates,geoData.type,global_id
0,"125009, г. Москва, Георгиевский пер., д. 1, ст...",Центральный административный округ,[{'ContactPhone': '(495) 212-91-14'}],Тверской район,[{'Email': 'reforma@timeclub24.ru'}],Time Club Re форма,25360,С Тверской поворот в арку. После входа варку и...,Уютные заведения с домашней обстановкой для ра...,Time Club Re форма,timeclub24.ru/,NaN,Переговорная комната-1190.00 Переговорная комн...,2012.0,"[37.6134492450171, 55.758981526325776]",Point,4066284
1,"105077, г. Москва, ул. Средняя Первомайская, д. 3",Восточный административный округ,[{'ContactPhone': '(495) 225-14-14'}],район Измайлово,[{'Email': 'info@mbm.ru'}],Коворкинг-центр ЦУБ ВАО,25348,NaN,В Коворкинг-центре ЦУБ ВАО предоставляются бес...,Коворкинг-центр ЦУБ ВАО,www.mbm.ru,NaN,Закрепленное рабочее место-0.00 Закрепленное р...,2012.0,"[37.803828912974446, 55.79583347364307]",Point,4066287


In [6]:
import re

free_rx = re.compile("\\b0\\.00\\b")
def is_free(tariff):
    return bool(free_rx.search(tariff))

tariffs = df.Tariffs[df.Tariffs.notnull()]
number_of_free = 0
for tariff in tariffs:
    if is_free(tariff):
        number_of_free += 1

print("Количество бесплатных коворкингов:", number_of_free)
print("Доля бесплатных коворкингов:", number_of_free / len(tariffs))

Количество бесплатных коворкингов: 11
Доля бесплатных коворкингов: 0.13095238095238096


In [5]:
money_rx = re.compile("\\b(\\d+\\.\\d\\d)\\b")
max_cost = 0
for tariff in tariffs:
    for cost in money_rx.findall(tariff):
        max_cost = max(max_cost, float(cost))

print("Максимальная стоимость услуги:", max_cost, "рублей")

Максимальная стоимость услуги: 250000.0 рублей


In [6]:
import requests
from urllib import parse as urlparse
from bs4 import BeautifulSoup, SoupStrainer


def normalize_url(url):
    if not url.startswith("http"):
        url = "http://" + url
    return url


def get_page_and_title(url):
    try:
        page = requests.get(normalize_url(url))
        soup = BeautifulSoup(page.text, "html.parser")
        return page.text, soup.title.string
    except:
        return "", ""


def is_relative(url):
    return not bool(urlparse.urlparse(url).netloc)


def is_external(url, site):
    host = urlparse.urlparse(url).netloc
    return bool(host) and (host not in site)

In [7]:
titles = list()
name_on_page_counts = list()
external_link_counts = list()
for site, name in zip(df.Site, df.FullName):
    p, t = get_page_and_title(site)
    titles.append(t)
    name_on_page_counts.append(len(re.findall(name, p)))
    soup = BeautifulSoup(p, "html.parser", parse_only=SoupStrainer("a"))
    ext_link_cnt = 0
    for tag in soup:
        if tag.has_attr("href"):
            link = tag["href"]
            if is_external(link, site):
                ext_link_cnt += 1
    external_link_counts.append(ext_link_cnt)

In [8]:
df = df.assign(SiteTitle=pd.Series(titles).values)

In [9]:
df = df.assign(NameCount=pd.Series(name_on_page_counts).values)

In [10]:
df = df.assign(ExternalLinkCount=pd.Series(external_link_counts).values)

In [11]:
df.head()

,Address,AdmArea,ContactPhone,District,Email,FullName,ID,LocationClarification,ShortDescription,ShortName,Site,Specialization,Tariffs,Year,geoData.coordinates,geoData.type,global_id,SiteTitle,NameCount,ExternalLinkCount
0,"125009, г. Москва, Георгиевский пер., д. 1, ст...",Центральный административный округ,[{'ContactPhone': '(495) 212-91-14'}],Тверской район,[{'Email': 'reforma@timeclub24.ru'}],Time Club Re форма,25360,С Тверской поворот в арку. После входа варку и...,Уютные заведения с домашней обстановкой для ра...,Time Club Re форма,timeclub24.ru/,NaN,Переговорная комната-1190.00 Переговорная комн...,2012.0,"[37.6134492450171, 55.758981526325776]",Point,4066284,Time Club — Сеть антикафе в Москве,0,13
1,"105077, г. Москва, ул. Средняя Первомайская, д. 3",Восточный административный округ,[{'ContactPhone': '(495) 225-14-14'}],район Измайлово,[{'Email': 'info@mbm.ru'}],Коворкинг-центр ЦУБ ВАО,25348,NaN,В Коворкинг-центре ЦУБ ВАО предоставляются бес...,Коворкинг-центр ЦУБ ВАО,www.mbm.ru,NaN,Закрепленное рабочее место-0.00 Закрепленное р...,2012.0,"[37.803828912974446, 55.79583347364307]",Point,4066287,ГБУ «Малый бизнес Москвы» — помощь малому бизнесу,0,5
2,"101000, г. Москва, ул. Мясницкая, д. 13, стр. 24",Центральный административный округ,[{'ContactPhone': '(495) 625-48-60'}],Красносельский район,[{'Email': 'krugozorclub@yandex.ru'}],Антикафе и коворкинг «Хорошая республика»,25347,NaN,\nХорошая республика рада видеть Вас в гостях...,Антикафе и коворкинг «Хорошая республика»,www.goodrepublic.ru/coworking/,NaN,Игровая комната-5000.00 Игровая комната; Перег...,2014.0,"[37.63407421967771, 55.763464355635584]",Point,4066291,"Коворкинг Хорошая Республика, GoodRepublic — А...",0,5
3,"127206, г. Москва, ул. Соломенной Сторожки, д....",Северный административный округ,[{'ContactPhone': '(495) 225-14-14'}],Тимирязевский район,[{'Email': 'info@mbm.ru'}],Коворкинг-центр ЦУБ САО,25349,NaN,В Коворкинг-центре ЦУБ САО предоставляются бес...,Коворкинг-центр ЦУБ САО,www.mbm.ru,NaN,Закрепленное рабочее место-0.00 Закрепленное р...,2014.0,"[37.56490512240495, 55.80879327409466]",Point,20737184,ГБУ «Малый бизнес Москвы» — помощь малому бизнесу,0,5
4,"129272, г. Москва, Рижская пл., д. 9, стр. 2",Северо-Восточный административный округ,[{'ContactPhone': '(985) 336-63-76'}],район Марьина Роща,[{'Email': 'admin@coworiga.ru'}],Коворкинг на Рижской,28006,Расположен в 5 минутах от метро и 10 минутах о...,"Грамотная концепция пространства, удобное рабо...",Коворкинг на Рижской,www.coworiga.ru/,NaN,"Закрепленное рабочее место (Тарифный план ""Баз...",NaN,"[37.631775482642716, 55.79774787923413]",Point,612740697,"Коворкинг в центре Москвы, аренда коворкинга н...",1,0
